## On this notebook we extract Censo 2010 individual data from their files.


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
import os

if not os.path.exists('./../../samplerCensoARG/'):
    !git clone https://github.com/matuteiglesias/samplerCensoARG.git ./../../

In [4]:
!python ./../../samplerCensoARG/samplear.py -dbp '/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub' -f 0.01 -y 2003 2023 -n ARG

/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/miglesia/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
['/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub']
[0.01]

In [6]:
import pandas as pd
# pd.read_csv('/home/miglesia/repositories/indice-pobreza-ExactasUBA/notebooks/../data/info/AGLO_rk/')

## Cargar info accesoria (Regiones, aglomerados, etc)

In [7]:
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')

radio_AGLO = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/Aglomerados-EPH-INDEC/main/result/radios_aglo_EPH.csv')
radio_AGLO['radio'] = radio_AGLO.COD_2010.str.replace('XX', '99').astype(int)
radio_AGLO['AGLOMERADO'] = radio_AGLO.eph_codagl
radio_AGLO['NOMAGLO'] = radio_AGLO.eph_aglome

radio_ref = radio_ref.drop(['AGLOMERADO'], axis = 1).merge(radio_AGLO[['radio','AGLOMERADO', 'NOMAGLO']], how = 'left')
radio_ref['AGLOMERADO'] = radio_ref['AGLOMERADO'].fillna(0).astype(int)

# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

In [58]:

AGLO_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/AGLO_rk')
rk_table = AGLO_rk.set_index(['ANO4', 'AGLOMERADO']).unstack()
AGLO_rk_filled = rk_table.fillna(rk_table.mean()).stack().reset_index()
AGLO_rk = AGLO_rk_filled

Reg_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/Reg_rk')


## Loop sampleo de censo

In [60]:
frac = 0.01 ## Frac needs to be the fraction used in the sampling (eg. -f 0.01 needs frac = 0.01)
startyr = 2003
endyr = 2023

for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    table = pd.read_csv('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv')
    table['ANO4'] = int(yr)
    
    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    ## VIVIENDA
    table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
    ## HOGAR
    table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
    table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
    table['H16'] = table['H16'].clip(0, 9)
    table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
    table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
    # PERSONA
    table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

    ## Agregar Region
    table = table.merge(dpto_region[['DPTO', 'Region']])

    ## Agregar ranking de Region y Aglo
    print(table.shape)
    table = table.merge(AGLO_rk[['AGLOMERADO', 'ANO4', 'AGLO_rk']]).merge(Reg_rk[['Region', 'ANO4', 'Reg_rk']])
    print(table.shape)
    
    table.to_csv('/media/miglesia/Elements/suite/yr_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv', index = False)  # Copias en carpeta yr_samples, en nuestra carpeta de indice de pobreza
    

2003
(364816, 38)
(364816, 40)
2004
(371771, 38)
(371771, 40)
2005
(376756, 38)
(376756, 40)
2006
(381953, 38)
(381953, 40)
2007
(388097, 38)
(388097, 40)
2008
(389856, 38)
(389856, 40)
2009
(399102, 38)
(399102, 40)
2010
(399704, 38)
(399704, 40)
2011
(404363, 38)
(404363, 40)
2012
(411680, 38)
(411680, 40)
2013
(415168, 38)
(415168, 40)
2014
(419343, 38)
(419343, 40)
2015
(425552, 38)
(425552, 40)
2016
(428908, 38)
(428908, 40)
2017
(435315, 38)
(435315, 40)
2018
(435109, 38)
(435109, 40)
2019
(441375, 38)
(441375, 40)
2020
(446156, 38)
(446156, 40)
2021
(453251, 38)
(453251, 40)
2022
(455576, 38)
(0, 40)
